# Example of a longer chain which creates a story, then translates it into another language

This idea came from the Complete LangChain & LLM's guide on Coursera 

In [5]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnableMap

# === Initialize Ollama LLM with gemma3:1b model ===
llm = OllamaLLM(model="gemma3:1b", temperature=0.7)

# === Prompt to generate a children's lullaby ===
story_prompt = PromptTemplate(
    input_variables=["location", "name"],
    template="""
As a children's book writer, please come up with a simple and short (90 words)
lullaby based on the location
{location}
and the main character {name}

STORY:
"""
)

# === Prompt to translate the story ===
translate_prompt = PromptTemplate(
    input_variables=["story", "language"],
    template="""
Translate the {story} into {language}. Make sure 
the language is simple and fun.

TRANSLATION:
"""
)

# === Create a Runnable pipeline for story generation ===
story_chain = story_prompt | llm

# === Create a Runnable pipeline for translation ===
# We'll dynamically feed the output of story_chain into this
translate_chain = translate_prompt | llm

# === Compose both chains into a sequential RunnableMap ===
overall_chain = RunnableMap({
    # First step: generate the story
    
    "story": lambda inputs: story_chain.invoke(inputs),
    
    # Second step: translate the story to the target language
    "translated": lambda inputs: translate_chain.invoke({
        "story": story_chain.invoke(inputs),
        "language": inputs["language"]
    })
})

# === Invoke the overall chain ===
response = overall_chain.invoke({
    "location": "Mordor",
    "name": "Cameron",
    "language": "French"
})

# === Output the results ===
print(f"English Version ====> {response['story']}\n")
print(f"Translated Version ====> {response['translated']}")


English Version ====> Okay, here's a short lullaby inspired by Mordor, aiming for a gentle, slightly melancholic feel, suitable for a child:

(Soft, slow melody begins)

“Mordor sleeps, a shadowed land,
Where purple hills forever stand.
The wind sighs low, a lonely plea,
For a little light, for you and me.

Cameron sleeps, beneath a stone,
His heart feels lost, all alone.
But stars like diamonds, softly gleam,
Dream of sunshine, a hopeful dream. 

Sleep now, little one, safe and deep,
Mordor’s secrets, you will keep.”

---

Would you like me to tweak this, perhaps adding a specific detail about Cameron’s situation or a comforting image?

Translated Version ====> Okay, here’s a translation of the lullaby, aiming for that gentle, melancholic, and slightly whimsical feel, with French additions and simplified language suitable for a child:

“Mordor dort, un pays sombre et noir,
Où la nuit grandit, et les ombres s’enroulent.
La pluie tombe doucement, une petite voix,
Pour un petit réconfort